In [1]:
from nltk.book import *
from nltk.metrics import edit_distance
import numpy as np
from itertools import combinations

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [2]:
def proccess(text):
    return set(
        [w.lower() for w in text if len(w) <= 7]
    )
    
pro = [list(proccess(t)) for t in [text1, text2, text3]]

In [3]:
edit_distance(pro[0][0], pro[0][1])
print(pro[0][0], pro[0][1])

vendome shooks


In [4]:
def cluster(words, nums):
    
    # each word has his own cluster
    clusters = np.array([i for i in range(len(words))])
    acc_nums = len(clusters)
    print(clusters)
    
    def merge_clusters(i,j):
        # i is bigger then j
        if i < j:
            i , j = j, i
        for idx in range(len(clusters)):
            if clusters[idx] == i:
                clusters[idx] = j
            elif clusters[idx] > i:
                clusters[idx] -= 1
                
    def find_closest_clusters():
        min_ed = 8
        candidates =  (0,0)
        for w1, w2 in combinations(enumerate(words), 2):            
            if clusters[w1[0]] != clusters[w2[0]]:
                ed = edit_distance(w1[1], w2[1])            
                if ed <= min_ed:                    
                    min_ed = ed
                    candidates = (clusters[w1[0]], clusters[w2[0]])
        return candidates
                    
            
    while acc_nums > nums:
        i,j = find_closest_clusters()
        merge_clusters(i,j)
        acc_nums -= 1
        print(clusters)
    print(acc_nums)
    return clusters
    

In [5]:
words = ["aa", "aaa", "bartek", "ania", "barte"]
cl = cluster(words, 2)

[0 1 2 3 4]
[0 1 2 3 2]
[0 0 1 2 1]
[0 0 1 0 1]
2


In [6]:
clustered = [[],[]]
for l in range(2):
    for c in range(len(cl)):
        if cl[c] == l:
            clustered[l].append(words[c])

In [7]:
clustered

[['aa', 'aaa', 'ania'], ['bartek', 'barte']]

In [8]:
words = list(set(pro[0] + pro[1] + pro[2]))
# dont even try


In [9]:
def naive_cluster(words, nums):
    clusters = sorted(words)
    cl_leng = len(words)//nums
    cluster = np.ndarray(len(words), dtype = np.int)
    for i in range(nums):
        for j in range(i*cl_leng, (i+1)*cl_leng):
            cluster[j] = i
    if len(words)%nums != 0:
        cluster[-(len(words)%nums):] = nums-1
    return (clusters, cluster)

In [10]:
words = ["ala", "tom", "bartek", "ania", "bart", "zbyszek"]
naive_cluster(words, 3)

(['ala', 'ania', 'bart', 'bartek', 'tom', 'zbyszek'],
 array([0, 0, 1, 1, 2, 2]))

In [11]:
[1,2,3,4,5][-2:]

[4, 5]

In [12]:
def final_version(words, nums):
    # to pick candidates for most far away points
    words_sort = sorted(words, key = lambda x: len(x))
    centered = []
    for i in range(nums):
        centered.append(words_sort[i*len(words)//nums])
    clusters = np.zeros(len(words), dtype=np.int)
    for i in range(len(words)):
        w = words[i]
        min_ed = 8
        cluster = 0
        for c in centered:
            ed = edit_distance(w, c)
            if ed <= min_ed:
                min_ed = ed
                clusters[i] = centered.index(c)
                cluster = centered.index(c)
    return (centered, clusters)
        


In [13]:
words = ["ala", "tom", "bartek", "ania", "bart", "zbyszek"]
final_version(words, 3)

(['ala', 'ania', 'bartek'], array([0, 0, 2, 1, 2, 2]))

In [19]:
def cluster_sizes(clusters):
    unique, counts = np.unique(clusters, return_counts=True)
    return dict(zip(unique, counts))

In [14]:
words = list(set(pro[0] + pro[1] + pro[2]))

In [15]:
res2 = final_version(words, 2)


In [16]:
res3 = final_version(words, 3)

In [17]:
res10 = final_version(words, 10)

In [20]:
cluster_sizes(res2[1])

{0: 3106, 1: 7653}

In [21]:
cluster_sizes(res3[1])

{0: 1369, 1: 4932, 2: 4458}

In [22]:
cluster_sizes(res10[1])

{0: 515,
 1: 1123,
 2: 960,
 3: 883,
 4: 2110,
 5: 550,
 6: 1661,
 7: 1324,
 8: 691,
 9: 942}